#Programa que muestra las consultas para hallar de manera indirecta la masa de las 591 estrellas

## Instalamos los paquetes que nos permiten consultar en Gaia

In [ ]:
pip install astropy

In [ ]:
pip install astro-gala astroquery

     |████████████████████████████████| 12.5 MB 4.2 MB/s 
     |████████████████████████████████| 4.5 MB 28.5 MB/s 
     |████████████████████████████████| 832 kB 37.7 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 3.6 MB 10.0 MB/s 


In [ ]:
pip install gala

     |████████████████████████████████| 10.6 MB 5.4 MB/s 


## Cargamos los paquetes

In [ ]:
import pandas as pd

# astropy imports
import astropy.coordinates as coord
from astropy.table import QTable
import astropy.units as u
from astroquery.gaia import Gaia

# Third-party imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# gala imports
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

# Montamos el archivo con las 591 estrellas para consultar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
datos3 = pd.read_csv('/content/drive/MyDrive/ASTRONOMÍA/Seminario_Trabajo_Grado_HVS/lamostdr7_gaiadr2_hvs_591_sin_DR2.txt', header=0, delimiter = ',')

In [ ]:
datos3.to_csv('/content/drive/MyDrive/ASTRONOMÍA/Seminario_Trabajo_Grado_HVS/lamostdr7_gaiadr2_hvs_591_sin_DR2.csv')

In [ ]:
datos3.head(4)

## Toma los valores de la columna que vamos a consultar del DataFrame y los organiza en tupla

In [ ]:
data = np.array(datos3['Gaia_designation'].values)
data2 =tuple(data)

## Hacemos la Consulta (query) para DR2

In [ ]:
query_text = f'''SELECT source_id, ra, dec, parallax, pmra, pmdec, 
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, radial_velocity, radius_val,radius_percentile_lower,radius_percentile_upper
FROM gaiadr2.gaia_source 
WHERE (gaiadr2.gaia_source.source_id IN {data2})'''

## También para el EDR3 (si se pudiera)

No está disponible todavía el campo radius_val en el EDR3

https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html

In [ ]:
"""
query_text = f'''SELECT designation source_id, ra, dec, parallax, pmra, pmdec, 
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, dr2_radial_velocity, radius_val,radius_percentile_lower,radius_percentile_upper
FROM gaiaedr3.gaia_source 
WHERE (gaiaedr3.gaia_source.designation IN {data2})'''
"""

## Ahora organizamos los resultados con ayuda de Gaia

In [ ]:
job = Gaia.launch_job(query_text)
gaia_data = job.get_results()

print(f"results = {gaia_data}")
gaia_data.write('gaia_data.csv')

### Generamos un archivo csv y lo miramos

In [ ]:
gaia_data = QTable.read('gaia_data.csv') #lo guardamos con extensión csv porque en .fits no es conveniente para lo que queremos

In [ ]:
gaia_data[:]

In [ ]:
"""dist = coord.Distance(parallax=u.Quantity(gaia_data['parallax']))
dist.min(), dist.max()"""

Esto último funciona si dejamos el archivo generado con extensión .fits

## Ahora vamos a cargar el dataframe con las 591 estrellas y lo fusionamos con el archivo de Gaia

In [ ]:
#Ahora cargamos los datos
datos = pd.read_csv('/content/drive/MyDrive/ASTRONOMÍA/Seminario_Trabajo_Grado_HVS/lamostdr7_gaiadr2_hvs_591.csv', header=0, delimiter = ',')

In [ ]:
datos.columns

In [ ]:
cols_int = ['ID', 'specid', 'Gaia_designation', 'R.A.', 'decl.',\
       'S/N_r', 'Class', 'pmra', 'e_pmra','pmdec', 'parallax', 'G', 'G_BP', 'G_RP',\
       'astrometric_flag', 'RV_LASP_Calibrated', 'r', 'x', 'el_x', 'eu_x','y','el_y', 'eu_y', 'z','z', 'el_z', 'eu_z', 'r_GC','el_r_GC', 'eu_r_GC',\
       'Vx', 'Vy', 'Vz', 'V_GC', 'e', 'Zmax','r_min','el_r_min', 'eu_r_min',\
       'E_Phi_Infinity','Vesc(W)','Vesc(G)', 'Vesc(Ke)', 'Vesc(Ko)', 'Vesc(P)', 'Vesc(M)', 'Vesc(X)',\
       'Pgc', 'PMW', 'Pub(W)', 'Pub(G)', 'Pub(Ke)', 'Pub(Ko)', 'Pub(P)','Pub(M)', 'Pub(X)', 'Origin_Class', 'Teff_LASP', 'e_Teff_LASP',\
       'log(g)_LASP', 'e_log(g)_LASP', '[Fe/H]_LASP', 'e_[Fe/H]_LASP','Teff_DD-Payne', 'e_Teff_DD-Payne', 'log(g)_DD-Payne',\
       'e_log(g)_DD-Payne', '[Fe/H]_DD-Payne', 'e_[Fe/H]_DD-Payne',\
       'Alpha_Fe_DD-Payne', 'e_Alpha_Fe_DD-Payne', 'conservative_results' ]
df = datos[cols_int]
df.head()

# Este archivo de LAMOST pesa más de 2 Gigas, por tanto si se desea correr todo el Notebook se debe descargar directamente de http://www.lamost.org/lmusers/

In [ ]:
datos2 = pd.read_csv('/content/drive/MyDrive/ASTRONOMÍA/Seminario_Trabajo_Grado_HVS/dr6_med_v2_MRS.csv', header=0, delimiter = '|')

In [ ]:
datos2.columns

# Ahora hacemos la tabla cruzada con Gaia

In [ ]:
src_id = df["specid"].values
##df: tabla con 591
##df2: catálogo LAMOST
cross_LAMOST = datos2[datos2.planid.isin(src_id)].reset_index(drop=True)
print(cross_LAMOST)

In [ ]:
datos4 = pd.read_csv( '/content/gaia_data.csv')

In [ ]:
datos4.columns

In [ ]:
datos4.head(4)

## Usamos la columna source_id como referencia en los dos Datasets (Gaia y 591 estrellas)

In [ ]:
src_id = datos4["source_id"].values
##datos4 son los datos de consulta de Gaia
##datos3 son los datos de las 591 estrellas
cross_GAIA = datos3[datos3.Gaia_designation.isin(src_id)].reset_index(drop=True)
print(cross_GAIA)

## Se espera que todas las 591 estrellas aparezcan en la tabla cruzada con Gaia

In [ ]:
cross_GAIA.head(4)

In [ ]:
cross_GAIA.groupby(['Origin_Class']).count()

## Reorganizo las columnas para hacer el merge final

In [ ]:
#Creo una función que me reorganice las posiciones de la columna
def re_list(x,y):
  l1 = x.copy() #copia las columnas (x debe ser una lista)
  i1 = x.index(y) #me indexa la columna
  el1 = [l1[i1]] #la ubica
  l1.remove(el1[0]) #la elimina de la lista original
  return el1 +l1 #la agrega nuevamente de primera y le suman lo que queda de la original

In [ ]:
#aca necesito que los dos dataframes (datos3 y datos 4 tengan sus columnas como listas)
columna1 = list(datos3.columns)
columna2 = list(datos4.columns)

In [ ]:
columna2

In [ ]:
#Lo aplicamos a nuestros dos dataframes
datos5 = datos3[re_list(columna1,'Gaia_designation')]
print(datos5)

In [ ]:
datos6 = datos4[re_list(columna2,'source_id')]
datos6

In [ ]:
# Hacemos la última equivalencia entre dataframes para obtener uno
datos7 = datos6.rename(columns={'source_id': 'Gaia_designation'})
print(datos7)

In [ ]:
# Generamos el archivo fusionado con los valores del radio
fusion_datos = pd.merge(datos5,datos7[['Gaia_designation','radial_velocity','radius_val','radius_percentile_lower','radius_percentile_upper']],how="left")

In [ ]:
fusion_datos.head()

In [ ]:
# Finalmente obtenemos el archivo csv con la información de las masas obtenidas para HVS
fusion_datos.to_csv('/content/drive/MyDrive/ASTRONOMÍA/Seminario_Trabajo_Grado_HVS/591_HiVelSCs_Radius.csv', sep=',')